# Creating 1-Dimesional Audio Feature Vectors for Clustering 

This notebook borrows heavliy from the following three sources: 

- [Audio Signal Processing for Machine Learning](https://www.youtube.com/playlist?list=PL-wATfeyAMNqIee7cH3q1bh4QJFAaeNv0)
- [librosa](https://librosa.org/doc/main/index.html)
- [Audio Feature Extraction](https://devopedia.org/audio-feature-extraction)

In [1]:
from IPython.display import display, HTML, Latex
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

class StopExecution(Exception):
    def _render_traceback_(self):
        print("Process Terminated")

### Env Setup

In [2]:
import os 
import math
import numpy as np
import librosa.display

import matplotlib.pyplot as plt
import librosa

In [7]:
ROOT = "/home/ec2-user/tugboat_interval_ds"
TUGBOAT_PATH = os.path.join(ROOT, "tugboat")
NOISE_PATH = os.path.join(ROOT, "no_tugboat")

SAMPLING_RATE = 16000
FRAME_LENGTH = 2048
HOP_LENGTH = FRAME_LENGTH // 2
WINDOW_LENGTH = FRAME_LENGTH

INTERVAL_LENGTH = 10 # seconds
SPLIT_FREQUENCY = 2000

### Data Loading

In [4]:
tugboat_files_ls = [
    os.path.join(TUGBOAT_PATH, x)
    for x in os.listdir(TUGBOAT_PATH)
    if x.endswith(".wav")
]
noise_files_ls = [
    os.path.join(NOISE_PATH, x) for x in os.listdir(NOISE_PATH) if x.endswith(".wav")
]

In [5]:
len(tugboat_files_ls), len(noise_files_ls)

(4030, 2918)

### Window Features Driver Function

In [6]:
def create_and_save(
    file_ls,
    out_dir,
    feature_func,
    feature_interval,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
):
    target_ctr = 0
    for file in file_ls:
        signal, sampling_rate = librosa.load(file, sr=sampling_rate)
        if len(signal) > 0:
            feature = feature_func(
                signal, frame_length=frame_length, hop_length=hop_length
            )
            frames = range(0, feature.size)
            feature_time = librosa.frames_to_time(
                frames, hop_length=hop_length, sr=sampling_rate
            )
            prev_window = None
            prev_index = 0
            if len(feature_time) > 0:
                for window in range(0, math.ceil(feature_time[-1]), feature_interval):
                    # Throws away windows shorter than interval
                    if prev_window != None:
                        curr_index = len(
                            feature_time[
                                (feature_time < window) & (feature_time > prev_window)
                            ]
                        )
                        save_feature = feature[prev_index : prev_index + curr_index]
                        save_feature.flags.writeable = False
                        np.save(os.path.join(out_dir, str(target_ctr)), save_feature)
                        target_ctr += 1
                        prev_index = prev_index + curr_index

                    prev_window = window
    return f"Saved {target_ctr} features"

## Time Domain Audio Features

### Ampilitude Envelope 

Takes the max ampilitude for the window size you specify

In [7]:
AE_TUG_OUT = "/home/ec2-user/clustering/amplitude_envelope/tugboat"
AE_NOISE_OUT = "/home/ec2-user/clustering/amplitude_envelope/no_tugboat"

os.makedirs(AE_TUG_OUT, exist_ok=True)
os.makedirs(AE_NOISE_OUT, exist_ok=True)

In [8]:
def amplitude_envelope(signal, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH):
    return np.array(
        [max(signal[i : i + frame_length]) for i in range(0, len(signal), hop_length)]
    )

In [9]:
create_and_save(
    tugboat_files_ls,
    AE_TUG_OUT,
    amplitude_envelope,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 5261 features'

In [10]:
create_and_save(
    noise_files_ls,
    AE_NOISE_OUT,
    amplitude_envelope,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 169484 features'

'Saved 169484 features'

'Saved 169484 features'

### Root Mean Squared Energy (RMSE)

RMSE generally captures the average of the ampilitude changes within a given windows while minimizing outlier-related variance 

In [7]:
RMSE_TUG_OUT = "/home/ec2-user/clustering/rmse/tugboat"
RMSE_NOISE_OUT = "/home/ec2-user/clustering/rmse/no_tugboat"

os.makedirs(RMSE_TUG_OUT, exist_ok=True)
os.makedirs(RMSE_NOISE_OUT, exist_ok=True)

In [8]:
def rmse(signal, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH):
    return np.array(
        [
            np.sqrt(np.sum(signal[i : i + frame_length] ** 2) / frame_length)
            for i in range(0, len(signal), hop_length)
        ]
    )

In [9]:
create_and_save(
    tugboat_files_ls,
    RMSE_TUG_OUT,
    rmse,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 5261 features'

In [10]:
create_and_save(
    noise_files_ls,
    RMSE_NOISE_OUT,
    rmse,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 169484 features'

### Zero-Crossing Rate (ZCR)

ZCR measures the number of sign changes and so are typically used in speech detection because human speech contains many +/- ampilitude changes 

In [11]:
ZRC_TUG_OUT = "/home/ec2-user/clustering/zcr/tugboat"
ZRC_NOISE_OUT = "/home/ec2-user/clustering/zcr/no_tugboat"

os.makedirs(ZRC_TUG_OUT, exist_ok=True)
os.makedirs(ZRC_NOISE_OUT, exist_ok=True)

In [12]:
def zcr(signal, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH):
    return librosa.feature.zero_crossing_rate(
        signal, frame_length=frame_length, hop_length=hop_length
    ).flatten()

In [13]:
create_and_save(
    tugboat_files_ls,
    ZRC_TUG_OUT,
    zcr,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 5261 features'

In [14]:
create_and_save(
    noise_files_ls,
    ZRC_NOISE_OUT,
    zcr,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 169484 features'

## Frequency Domain Audio Features

### Spectral Centroid

A Magnitude spectrogram (Frequency Domain in Hz) is windowed and the center of gravity of the magnitude spectrum is extracted (where most of the energy is concentrated)

In [18]:
SC_TUG_OUT = "/home/ec2-user/clustering/spectral_centroid/tugboat"
SC_NOISE_OUT = "/home/ec2-user/clustering/spectral_centroid/no_tugboat"

os.makedirs(SC_TUG_OUT, exist_ok=True)
os.makedirs(SC_NOISE_OUT, exist_ok=True)

In [19]:
def spectral_centroid(
    signal,
    frame_length=FRAME_LENGTH,
    hop_length=HOP_LENGTH,
    sampling_rate=SAMPLING_RATE,
):
    return librosa.feature.spectral_centroid(
        y=signal, sr=sampling_rate, n_fft=frame_length, hop_length=hop_length
    ).flatten()

In [20]:
create_and_save(
    tugboat_files_ls,
    SC_TUG_OUT,
    spectral_centroid,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 5261 features'

In [21]:
create_and_save(
    noise_files_ls,
    SC_NOISE_OUT,
    spectral_centroid,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 169484 features'

### Spectral Bandwidth 

The variance of the spectral centroid that is proportional to the amount of energy in all of the bands

In [31]:
SB_TUG_OUT = "/home/ec2-user/clustering/spectral_bandwidth/tugboat"
SB_NOISE_OUT = "/home/ec2-user/clustering/spectral_bandwidth/no_tugboat"

os.makedirs(SB_TUG_OUT, exist_ok=True)
os.makedirs(SB_NOISE_OUT, exist_ok=True)

In [32]:
def spectral_bandwidth(
    signal,
    frame_length=FRAME_LENGTH,
    hop_length=HOP_LENGTH,
    sampling_rate=SAMPLING_RATE,
):
    return librosa.feature.spectral_bandwidth(
        y=signal, sr=sampling_rate, n_fft=frame_length, hop_length=hop_length
    ).flatten()

In [33]:
create_and_save(
    tugboat_files_ls,
    SB_TUG_OUT,
    spectral_bandwidth,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 5261 features'

In [34]:
create_and_save(
    noise_files_ls,
    SB_NOISE_OUT,
    spectral_bandwidth,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 169484 features'

### Band Energy Ratio

A ratio created by defining some frequency to divide the singal into two parts and then dividing the power in the higher frequency bins by the power in the lower frequency bins

In [8]:
BNE_TUG_OUT = "/home/ec2-user/clustering/band_energy_ratio/tugboat"
BNE_NOISE_OUT = "/home/ec2-user/clustering/band_energy_ratio/no_tugboat"

os.makedirs(BNE_TUG_OUT, exist_ok=True)
os.makedirs(BNE_NOISE_OUT, exist_ok=True)

In [9]:
def calcluate_split_frequency_bin(spectrogram, split_frequency, sample_rate):
    frequency_range = sample_rate / 2 
    frequency_delta_per_bin = frequency_range / spectrogram.shape[0]
    split_frequency_bin = np.floor(split_frequency / frequency_delta_per_bin)
    return int(split_frequency_bin)

def calculate_band_energy_ratio(spectrogram, split_frequency, sample_rate):
    split_frequency_bin = calcluate_split_frequency_bin(spectrogram, split_frequency, sample_rate)
    power_spec = np.abs(spectrogram) ** 2 
    power_spec = power_spec.T
    
    band_energy_ratio = []
    
    for frequencies_in_frame in power_spec:
        sum_power_low_frequencies = np.sum(frequencies_in_frame[:split_frequency_bin])
        sum_power_high_frequencies = np.sum(frequencies_in_frame[split_frequency_bin:])
        if sum_power_high_frequencies != 0:
            ber_current_frame = sum_power_low_frequencies / sum_power_high_frequencies
        else:
            ber_current_frame = 0
        band_energy_ratio.append(ber_current_frame)
    
    return np.array(band_energy_ratio)

def band_energy_ratio(
    signal,
    split_frequency=SPLIT_FREQUENCY,
    frame_length=FRAME_LENGTH,
    hop_length=HOP_LENGTH,
    sampling_rate=SAMPLING_RATE,
):
    specgram = librosa.stft(signal, n_fft=frame_length, hop_length=hop_length)
    return calculate_band_energy_ratio(specgram, split_frequency, sampling_rate)
    

In [10]:
create_and_save(
    tugboat_files_ls,
    BNE_TUG_OUT,
    band_energy_ratio,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 5261 features'

In [11]:
create_and_save(
    noise_files_ls,
    BNE_NOISE_OUT,
    band_energy_ratio,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 169484 features'

### Spectral Flux

Spectral Flux is a measure of how quickly the power spectrum of a signal is changing

In [57]:
SF_TUG_OUT = "/home/ec2-user/clustering/spectral_flux/tugboat"
SF_NOISE_OUT = "/home/ec2-user/clustering/spectral_flux/no_tugboat"

os.makedirs(SF_TUG_OUT, exist_ok=True)
os.makedirs(SF_NOISE_OUT, exist_ok=True)

In [63]:
def spectral_flux(
    signal,
    frame_length=FRAME_LENGTH,
    hop_length=HOP_LENGTH,
    sampling_rate=SAMPLING_RATE,
):
    return librosa.onset.onset_strength(y=signal, sr=sampling_rate, hop_length=hop_length)

In [64]:
create_and_save(
    tugboat_files_ls,
    SF_TUG_OUT,
    spectral_flux,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 5261 features'

In [65]:
create_and_save(
    noise_files_ls,
    SF_NOISE_OUT,
    spectral_flux,
    INTERVAL_LENGTH,
    sampling_rate=SAMPLING_RATE,
    hop_length=HOP_LENGTH,
    frame_length=FRAME_LENGTH,
)

'Saved 169484 features'